#### Apply Augmentations
We'll apply the following types of augmentations to the images:

##### a. Geometric Augmentations
Rotation: Simulate tilted text by rotating the image slightly.
Skewing: Introduce perspective distortions to mimic handwritten text scanned at an angle.
Scaling: Increase or decrease the size of text.
##### b. Color Augmentations
Brightness and Contrast: Simulate different lighting conditions or faded text.
Noise: Add Gaussian noise to mimic imperfections in scanned documents.
##### c. Text Variability
Ink Spread: Simulate smudged or faded ink.
Smudges: Add random smudges around text to mimic real handwriting.
##### d. Background Augmentations
Textured Backgrounds: Add backgrounds that resemble paper textures (e.g., lined paper, wrinkled paper, etc.).
Random Noise: Mimic imperfections in the paper or scanning process.

In [15]:
import os
import random
from imgaug import augmenters as iaa
from PIL import Image
import numpy as np
import cv2

In [16]:

# Paths
input_dir = "../artifacts/rendered_images"  # Folder containing the generated images
augmented_dir = "../artifacts/augmented_outputs"  # Folder to save augmented images

In [17]:

# Define augmentation pipeline with additional features
augmentation_pipeline = iaa.Sequential([
    iaa.Fliplr(0.5),  # 50% chance to flip horizontally
    iaa.Flipud(0.2),  # 20% chance to flip vertically
    iaa.Affine(
        rotate=(-25, 25),  # Rotate between -25 and 25 degrees
        scale=(0.8, 1.2)   # Scale image between 80% and 120%
    ),
    iaa.PerspectiveTransform(scale=(0.01, 0.15)),  # Simulate different angles
    iaa.AdditiveGaussianNoise(scale=(0, 0.05 * 255)),  # Add noise
    iaa.GaussianBlur(sigma=(0.0, 1.5)),  # Apply Gaussian blur
    iaa.Multiply((0.8, 1.2)),  # Adjust brightness
    iaa.LinearContrast((0.8, 1.2)),  # Adjust contrast
    iaa.OneOf([
        iaa.pillike.EnhanceSharpness((0.5, 2.0)),  # Adjust sharpness
        iaa.pillike.EnhanceContrast((0.5, 2.0))   # Adjust contrast dynamically
    ]),
    iaa.Sometimes(0.3, iaa.Dropout(p=(0.01, 0.05))),  # Occasional missing pixels (for realism)
])

In [18]:
# Additional augmentation functions
def add_coffee_stain(image_np):
    """Add occasional coffee stains or smudges for realism."""
    stain = iaa.Sometimes(0.3, iaa.BlendAlphaSimplexNoise(iaa.Multiply((0.5, 0.9)), per_channel=True))
    return stain(image=image_np)

def vary_pen_pencil_thickness(image_np):
    """Vary pen/pencil thickness and color to simulate different writing styles."""
    stroke_augmenter = iaa.OneOf([
        iaa.Sometimes(0.5, iaa.AddToHueAndSaturation((-30, 30))),  # Vary color tone
        iaa.Sometimes(0.7, iaa.GaussianBlur(sigma=(0.5, 1.5)))  # Simulate pen strokes
    ])
    return stroke_augmenter(image=image_np)

# Function to apply augmentations and save images
def augment_image(image_path, output_dir, augmentations, num_augmentations=5):
    """Apply augmentations including perspective, stains, and stroke variations."""
    image = Image.open(image_path).convert("RGB")  # Ensure image is RGB
    image_np = np.array(image)  # Convert to NumPy array

    base_name = os.path.splitext(os.path.basename(image_path))[0]  # Extract filename

    # Apply augmentations
    for i in range(num_augmentations):
        augmented_image_np = augmentations.augment_image(image_np)  # Apply augmentation
        augmented_image_np = augmentations(image=image_np)  # Apply augmentation
        augmented_image_np = add_coffee_stain(augmented_image_np)  # Add stains/smudges
        augmented_image_np = vary_pen_pencil_thickness(augmented_image_np)  # Vary writing instrument
        
        augmented_image = Image.fromarray(augmented_image_np)  # Convert back to image

        # Save the augmented image
        augmented_image.save(os.path.join(output_dir, f"{base_name}_aug_{i}.png"))

# Process all images in the input directory
for image_file in os.listdir(input_dir):
    if image_file.endswith(".png") or image_file.endswith(".jpg"):
        image_path = os.path.join(input_dir, image_file)
        augment_image(image_path, augmented_dir, augmentation_pipeline)

print("Augmentation complete. Augmented images saved in:", augmented_dir)

Augmentation complete. Augmented images saved in: ../artifacts/augmented_outputs
